# Heroes of Pymoli - Data Crunching

Here are some observations based on the customer data given:<br>
    1.  Despite the large difference in the amount of purchasing data in either data set, the average purchase price among all customers is almost exactly the same, $2.93 vs $2.92.
 
    2.  Males, accounting for over 81% of the population in both data sets, also made up over 81% of the total revenue. The highest normalized purchase value belongs to the Other / Non-Disclosed group at $4.47.
    
    3. In both data sets, the largest source of the game's revenue comes from the 18 - 22 year old and 22 - 26 year old groups.  However, the highest average purchase belongs to the 38 - 42 year old group (34-38 is the highest, then the 38-42 in the second data set). While they do not make as many purchases as the 18 - 26 year olds, their average purchase is worth $1.47 more in the first data set, and nearly 50 cents more in the second.


In [776]:
import pandas as pd
import os
#read in the first data file
game_data = pd.read_json('purchase_data.json')

# Raw Data

In [777]:
game_data.head(10) #display the first 10 rows of data

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


# Player Count

In [778]:
player_count = game_data['SN'].nunique() #gives the unique player count based on the Screenname column
pc_dict = {'Total Players':[player_count]}
player_count_df = pd.DataFrame(data=pc_dict,index=None)
player_count_df

,Total Players
0,573


# Purchasing Analysis (Total)

In [779]:
unique_items = game_data['Item ID'].nunique() #gives the unique item count based on the item ID column
total_purchases = game_data['Price'].count() #total count of purchases based on the price column
total_revenue = game_data['Price'].sum()
uniq_rev_dict = {'Number of Unique Items':[unique_items],'Average Price':['\$' + str(round(total_revenue/total_purchases,2))],'Number of Purchases':[total_purchases],'Total Revenue':['\$' + str(round(total_revenue,2))]}
uniq_rev_df = pd.DataFrame(data=uniq_rev_dict,index=None)
uniq_rev_df

,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,\$2.93,780,183,\$2286.33


# Gender Demographics

In [780]:
people_data = game_data.groupby('Gender') #Total number of male customers in data set. 
total_male = people_data.count()
male_num = total_male['Age']['Male']

uniq_names = game_data.drop_duplicates('SN',keep='first') 
uniq_male = uniq_names[uniq_names['Gender'] == 'Male'] #gives number of unique male customers

total_fem = total_male['Age']['Female'] #total number of female customers
uniq_fem = uniq_names[uniq_names['Gender'] == 'Female'] #total number of unique female customers

total_peep = total_male['Age']['Other / Non-Disclosed'] #total number of Other-Non/Disclosed purchasers
uniq_peep = uniq_names[uniq_names['Gender'] == 'Other / Non-Disclosed'] #total number of unique O/ND

#percentage breakdowns by gender
per_male = (uniq_male['SN'].count())/(uniq_male['SN'].count() + uniq_fem['SN'].count() + uniq_peep['SN'].count())
per_female = (uniq_fem['SN'].count())/(uniq_male['SN'].count() + uniq_fem['SN'].count() + uniq_peep['SN'].count())
per_peeps = (uniq_peep['SN'].count())/(uniq_male['SN'].count() + uniq_fem['SN'].count() + uniq_peep['SN'].count())

people_count = pd.DataFrame(columns = ['Percentage of Players','Total Count'],index = ['Male','Female','Other / Non-Disclosed'])
people_count['Percentage of Players']['Male'] = (str(round((per_male * 100),2)) + '%')
people_count['Percentage of Players']['Female'] = (str(round((per_female * 100),2)) + '%')
people_count['Percentage of Players']['Other / Non-Disclosed'] = (str(round((per_peeps * 100),2)) + '%')
people_count['Total Count']['Male'] = uniq_male['SN'].count()
people_count['Total Count']['Female'] = uniq_fem['SN'].count()
people_count['Total Count']['Other / Non-Disclosed'] = uniq_peep['SN'].count()
people_count

,Percentage of Players,Total Count
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclosed,1.4%,8


# Purchasing Analysis (Gender)

In [781]:
#purchase data for males
male_spend = game_data.groupby('Gender')
tot_spend = male_spend['Price'].sum()
m_spend = tot_spend['Male']
avg_m_spend = m_spend / male_num

#purchase data for females
f_spend = tot_spend['Female']
avg_f_spend = f_spend / total_fem

#purchase data for O/ND
p_spend = tot_spend['Other / Non-Disclosed']
avg_o_spend = p_spend / total_peep

purchase_data = pd.DataFrame(columns = ['Purchase Count','Average Purchase Price','Total Purchase Value','Normalized Totals'],index = ['Female','Male','Other / Non-Disclosed'])
purchase_data['Purchase Count']['Female'] = total_male['Age']['Female']
purchase_data['Purchase Count']['Male'] = total_male['Age']['Male']
purchase_data['Purchase Count']['Other / Non-Disclosed'] = total_male['Age']['Other / Non-Disclosed']
purchase_data['Average Purchase Price']['Female'] = ( '\$' + str(round(avg_f_spend,2)))
purchase_data['Average Purchase Price']['Male'] = ( '\$' + str(round(avg_m_spend,2)))
purchase_data['Average Purchase Price']['Other / Non-Disclosed'] = ( '\$' + str(round(avg_o_spend,2)))
purchase_data['Total Purchase Value']['Female'] = ( '\$' + str(round(float(f_spend), 2)))
purchase_data['Total Purchase Value']['Male'] = ( '\$' + str(round(float(m_spend), 2)))
purchase_data['Total Purchase Value']['Other / Non-Disclosed'] = ( '\$' + str(round(float(p_spend), 2)))
purchase_data['Normalized Totals']['Female'] = ( '\$' + str(round(f_spend/(uniq_fem['SN'].count()),2)))
purchase_data['Normalized Totals']['Male'] = ( '\$' + str(round(m_spend/(uniq_male['SN'].count()),2)))
purchase_data['Normalized Totals']['Other / Non-Disclosed'] = ( '\$' + str(round(p_spend/(uniq_peep['SN'].count()),2)))
purchase_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Female,136,\$2.82,\$382.91,\$3.83
Male,633,\$2.95,\$1867.68,\$4.02
Other / Non-Disclosed,11,\$3.25,\$35.74,\$4.47


# Age Demographics

In [782]:
bins = [0,10,14,18,22,26,30,34,38,42,46] #binning data by Age
age_ranges = ['0-10','10-14','14-18','18-22','22-26','26-30','30-34','34-38','38-42','42-46']
range_ages = pd.cut(game_data['Age'], bins, labels = age_ranges)
age_bin_list = range_ages.value_counts(ascending = False).sort_index()

age_demo = pd.DataFrame(columns = ['Percentage of Players','Total Count'],index=age_ranges)
uniq_range_ages = pd.cut(uniq_names['Age'], bins, labels = age_ranges)
uniq_age_bin_list = uniq_range_ages.value_counts(ascending = False).sort_index()
for i in range(0,10):
    age_demo['Percentage of Players'][age_demo['Percentage of Players'].index[i]] = round(((uniq_age_bin_list[i]/uniq_age_bin_list.sum()) * 100),2)
    age_demo['Total Count'][age_demo['Total Count'].index[i]] = uniq_age_bin_list[i]
age_demo

,Percentage of Players,Total Count
0-10,3.84,22
10-14,3.49,20
14-18,14.66,84
18-22,31.06,178
22-26,26.7,153
26-30,7.68,44
30-34,5.93,34
34-38,4.36,25
38-42,1.92,11
42-46,0.35,2


# Purchasing Analysis (Age)

In [783]:
game_data = game_data.assign(Bin = range_ages) #assign the Bin ranges to the data set

age_stats = game_data.groupby('Bin')
bin_spend_average = age_stats['Price'].mean() #mean purchase price for each Age group

bin_spend_total = age_stats['Price'].sum() #total spent by each Age group

uniq_names = uniq_names.assign(Bin = range_ages)
uniq_age_stats = uniq_names.groupby('Bin')
bin_spend_count = uniq_age_stats['Price'].count() #count of unique customers in each Age group

age_df = pd.DataFrame(columns = ['Purchase Count','Average Purchase Price','Total Purchase Value','Normalized Totals'], index = age_ranges)
for i in range(0,10):  
    age_df['Purchase Count'][age_df['Purchase Count'].index[i]] = age_bin_list[i]
    age_df['Average Purchase Price'][age_df['Average Purchase Price'].index[i]] = ('\$' + str(round(bin_spend_total[i]/age_bin_list[i],2)))
    age_df['Total Purchase Value'][age_df['Total Purchase Value'].index[i]] = ('\$' + str(round(bin_spend_total[i],2)))
    age_df['Normalized Totals'][age_df['Normalized Totals'].index[i]] = ('\$' + str(round(bin_spend_total[i]/bin_spend_count[i],2)))
age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0-10,32,\$3.02,\$96.62,\$4.39
10-14,31,\$2.7,\$83.79,\$4.19
14-18,111,\$2.88,\$319.32,\$3.8
18-22,231,\$2.93,\$676.2,\$3.8
22-26,207,\$2.94,\$608.02,\$3.97
26-30,63,\$2.98,\$187.99,\$4.27
30-34,46,\$3.07,\$141.24,\$4.15
34-38,37,\$2.81,\$104.06,\$4.16
38-42,20,\$3.13,\$62.56,\$5.69
42-46,2,\$3.26,\$6.53,\$3.26


# Top Spenders

In [784]:
top_spenders = game_data.sort_values('Price', ascending = False).head() #returns list of customers sorted by price

sn_spend = game_data.groupby('SN')
most_spend = sn_spend['Price'].sum() #sums the price column and filters by Screenname
top_5_spend = most_spend.sort_values(ascending = False).head()

best_cust_data = pd.DataFrame()
for i in range(5): #returns a DataFrame containing the customers who made the most number of purchases
    best_cust =(game_data['SN'] == top_5_spend.index[i])
    best_cust_1 = pd.DataFrame(game_data[best_cust])
    best_cust_data = best_cust_data.append(best_cust_1)
    
#Table containing highest spending customers
best_cust_tab = pd.DataFrame()
for i in range(5):
    best_cust_list = [{'Username':top_5_spend.index[i],'Total Purchases':best_cust_data['Price'][best_cust_data['SN']==top_5_spend.index[i]].count(),'Average Purchase':('\$' + str(round(best_cust_data['Price'][best_cust_data['SN']==top_5_spend.index[i]].mean(),2))),'Total Purchase Value':('\$' + str(round(best_cust_data['Price'][best_cust_data['SN']==top_5_spend.index[i]].sum(),2)))}]
    best_cust_tab = best_cust_tab.append(best_cust_list)
best_cust_tab[best_cust_tab.columns[::-1]]
best_cust_tab.set_index('Username')

,Average Purchase,Total Purchase Value,Total Purchases
Username,,,
Undirrala66,\$3.41,\$17.06,5
Saedue76,\$3.39,\$13.56,4
Mindimnya67,\$3.18,\$12.74,4
Haellysu29,\$4.24,\$12.73,3
Eoda93,\$3.86,\$11.58,3


# Most Popular Items

In [785]:
pop_items_tot = game_data.groupby('Item ID')
pop_items_data = pop_items_tot['Item ID'].count()
pop_items_new = pop_items_data.sort_values(ascending = False).head() #series of the most popular items, sorted by count of Item ID

#creates a DataFrame containing data for the most frequently bought items
popular_items = pd.DataFrame()
for i in range(5):
    popular_items_data = (game_data['Item ID'] == pop_items_new.index[i])
    pop_items_df = pd.DataFrame(game_data[popular_items_data])
    popular_items = popular_items.append(pop_items_df)

popular_items['Item Name'].astype('category')
pop_items = popular_items['Item Name'].unique() #sends the unique names of items into a new list

#creates a DataFrame that summarizes the most popular item data using the series of popular item names and the popular items data frame
popular_items_list = pd.DataFrame()
for i in range(5):
    pop_items_list = [{'Item ID':pop_items_new.index[i],'Item Name':pop_items[i],'Purchase Count':pop_items_new.values[i],'Item Price':('\$' + str(round(popular_items['Price'][popular_items['Item ID']==pop_items_new.index[i]].mean(),2))),'Total Value':('\$' + str(round(popular_items['Price'][popular_items['Item ID'] == pop_items_new.index[i]].sum(),2)))}]
    popular_items_list = popular_items_list.append(pop_items_list)

popular_items_list.set_index('Item ID')

,Item Name,Item Price,Purchase Count,Total Value
Item ID,,,,
84,Arcane Gem,\$2.23,11,\$24.53
39,"Betrayal, Whisper of Grieving Widows",\$2.35,11,\$25.85
31,Trickster,\$2.07,9,\$18.63
34,Retribution Axe,\$4.14,9,\$37.26
175,Woeful Adamantite Claymore,\$1.24,9,\$11.16


# Most Profitable Items

In [786]:
expen_items = game_data.groupby('Item ID')
expen_items_series = expen_items['Price'].sum().sort_values(ascending = False) #a series of the sum of the prices of the most profitable items, sorted by Item ID

#creates a DataFrame containing data on the most profitable items
expensive_items = pd.DataFrame()
for i in range(5):
    expensive_items_data = (game_data['Item ID'] == expen_items_series.index[i])
    expensive_items_df = pd.DataFrame(game_data[expensive_items_data])
    expensive_items = expensive_items.append(expensive_items_df)

expnitems = expensive_items['Item Name'].unique() #assigns all of the profitable item names to a Series

#creates a DataFrame containing the profitable item data
expen_items_df = pd.DataFrame()
for i in range(5):
    expen_items_data = [{'Item ID':expen_items_series.index[i],'Item Name':expnitems[i], 'Purchase Count':game_data['Item Name'][game_data['Item ID']==expen_items_series.index[i]].count(),'Item Price':('\$' + str(round(game_data['Price'][game_data['Item ID']==expen_items_series.index[i]].mean(),2))),'Total Purchase Value':('\$' + str(round(game_data['Price'][game_data['Item ID']==expen_items_series.index[i]].sum(),2)))}]
    expen_items_df = expen_items_df.append(expen_items_data)

expen_items_df.set_index('Item ID')

,Item Name,Item Price,Purchase Count,Total Purchase Value
Item ID,,,,
34,Retribution Axe,\$4.14,9,\$37.26
115,Spectral Diamond Doomblade,\$4.25,7,\$29.75
32,Orenmir,\$4.95,6,\$29.7
103,Singed Scalpel,\$4.87,6,\$29.22
107,"Splitter, Foe Of Subtlety",\$3.61,8,\$28.88
